In [46]:
import mbuild as mb
import numpy as np

In [47]:
class CY5(mb.Compound):
    def __init__(self):
        super(CY5, self).__init__(name="CY5")
        self.add(mb.load("CY5_backbone.mol2"))
        self.attachment_sites = {
            "left": {"nitrogen": self[22], "hydrogen": self[23]},
            "right": {"nitrogen": self[14], "hydrogen": self[35]}
        }
        self.capping_groups = {}

    def attachment_vector(self, site):
        head = self.attachment_sites[site]["hydrogen"]
        tail = self.attachment_sites[site]["nitrogen"]
        vec = head.pos - tail.pos
        return vec / np.linalg.norm(vec)

    def get_fixed_particles(self, site):
        group = self.capping_groups[site]
        return [p for p in group.particles()]
    
    def add_attachment_chain(self, site, chain_length, bond_length=0.145):
        hydrogen = self.attachment_sites[site]["hydrogen"]
        nitrogen = self.attachment_sites[site]["nitrogen"]
        last_bonding_atom = nitrogen
        for i in range(chain_length):
            if i != chain_length - 1:
                new_group = mb.lib.moieties.CH2()
            else:
                new_group = mb.lib.moieties.CH3()
                self.capping_groups[site] = new_group
            new_group.translate_to(nitrogen.pos)
            new_group.translate(by=self.attachment_vector(site) * bond_length * (i + 1))
            self.add(new_group)
            self.add_bond([new_group[0], last_bonding_atom])
            last_bonding_atom = new_group[0]
        self.remove(hydrogen)

In [48]:
cy5_backbone = CY5()
cy5_backbone.add_attachment_chain(site="left", chain_length=5)
cy5_backbone.add_attachment_chain(site="right", chain_length=5)
cy5_backbone.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [50]:
cy5_backbone.get_fixed_particles("left")

[<C pos=([-0.5692  0.5055 -0.477 ]), 4 bonds, id: 125938939602576>,
 <H pos=([-0.4622  0.5055 -0.477 ]), 1 bonds, id: 125938939602768>,
 <H pos=([-0.6049  0.5824 -0.4117]), 1 bonds, id: 125938939602960>,
 <H pos=([-0.6049  0.5236 -0.5763]), 1 bonds, id: 125938939603152>]

3Dmol.js failed to load for some reason. Please check your browser console for error messages.